# Figure 4 - Analysis

In [ ]:
import sys
sys.path.append("..")

from main import *
from visualization import *

import matplotlib.pyplot as plt
from brainspace.gradient.embedding import DiffusionMaps
from scipy.stats import pearsonr, spearmanr
from scipy.stats import zscore
from matplotlib.colors import LogNorm

plt.rcParams['font.size'] = 6

if torch.cuda.is_available():  
    device = "cuda:0" 
else:  
    device = "cpu" 
print(device)

# Part 1: Filtering noise in ellipse geometry

In [ ]:
N = 2500 # Number of nodes
sigma = 0.05 # Filtering kernel width

In [ ]:
vertices = np.load('../Files/vertices_ellipse.npy').astype('float')

order = np.argsort(vertices[:, 2]) # Ordering vertices along z-axis for more structured matrices
vertices = vertices[order]
eigenmodes = np.load('../Files/eigenmodes_ellipse.npy')[order]

random_ids = [1] * N + [0] * (vertices.shape[0] - N)
np.random.shuffle(random_ids)
coords = vertices[np.array(random_ids) == 1] # Node coordinates are taken randomly from 3D mesh vertices

Generating random timeseries and smoothing them spatially.

In [ ]:
timeseries = np.random.normal(0, 1, (N, 10000))
for i in range(timeseries.shape[0]):
    timeseries[i] = timeseries[i]
timeseries = zscore(timeseries, axis=1)

In [ ]:
timeseries_smoothed = zscore(spatial_smoothing(timeseries, coords, sigma=sigma), axis=1)

In [ ]:
plt.figure(figsize=(10, 4))
plt.imshow(timeseries[:, :1000], cmap='magma', aspect='auto', vmin=-1, vmax=2)

In [ ]:
plt.figure(figsize=(10, 4))
plt.imshow(timeseries_smoothed[:, :1000], cmap='magma', aspect='auto', vmin=-1, vmax=2)

Computing correlations

In [ ]:
C = torch.corrcoef(torch.tensor(timeseries)).numpy()
C_smoothed = torch.corrcoef(torch.tensor(timeseries_smoothed)).numpy()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].imshow(np.abs(C) + 1, cmap='magma', norm=LogNorm())
ax[1].imshow(np.abs(C_smoothed) + 1, cmap='magma', norm=LogNorm())

Computing geometric mode correspondence

In [ ]:
N_modes = 50

map = DiffusionMaps(n_components=N_modes)
modes1 = map.fit_transform(np.abs(C)).T

map = DiffusionMaps(n_components=N_modes, alpha=0)
modes2 = map.fit_transform(np.abs(C_smoothed)).T

In [ ]:
modes_geometric = eigenmodes[np.array(random_ids) == 1, :].T # Subsampling vertices, where network nodes are located
modes_geometric = modes_geometric[1:, :] # Excluding first mode (artifact)

In [ ]:
corrs1, mapping1 = compute_mode_similarity_matrix(modes_geometric[:modes1.shape[0]], modes1, return_mapping=True)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
cax = plt.imshow(np.abs(corrs1), cmap='Reds', vmin=0, vmax=1)
plt.xlabel('Functional modes')
plt.ylabel('Geometric modes')
#cbar = fig.colorbar(cax, ax=ax, fraction=0.04, pad=0.02)
plt.colorbar(cax, ax=ax, fraction=0.045, pad=0.02)
#plt.savefig('heart_modes_comparison.png')

In [ ]:
corrs2, mapping2 = compute_mode_similarity_matrix(modes_geometric[:modes2.shape[0]], modes2, return_mapping=True)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
cax = plt.imshow(np.abs(corrs2), cmap='Reds', vmin=0, vmax=1)
plt.xlabel('Functional modes')
plt.ylabel('Geometric modes')
#cbar = fig.colorbar(cax, ax=ax, fraction=0.04, pad=0.02)
plt.colorbar(cax, ax=ax, fraction=0.045, pad=0.02)
#plt.savefig('heart_modes_comparison.png')

Saving results for figure later.

In [ ]:
np.save('../Results/figure4_ellipse_coords.npy', coords)
np.save('../Results/figure4_timeseries_noise.npy',timeseries)
np.save('../Results/figure4_timeseries_smoothed.npy',timeseries_smoothed)
np.save('../Results/figure4_correlations_noise.npy', C)
np.save('../Results/figure4_correlations_smoothed.npy', C_smoothed)
np.save('../Results/figure4_mode_mapping_noise.npy', corrs1)
np.save('../Results/figure4_mode_mapping_smoothed.npy', corrs2)

# Plotting and saving 3D gradients for figure

In [ ]:
%matplotlib inline

In [ ]:
figs_noise, figs_smooth = [], []

Gray ellipse nodes

In [ ]:
%matplotlib inline

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, figsize=(5, 5), dpi=300)

ax.scatter(coords[:, 0], coords[:, 1], coords[:, 2], color=[0.5, 0.5, 0.5], s=5, alpha=0.2, edgecolor='None', zorder=-10)
ax.set_xlim([-0.25, 0.75])
ax.set_ylim([-0.25, 0.75])
ax.set_zlim([0, 1])
    
ax.set_axis_off()

plt.tight_layout(pad=0)

fig_array = figure_to_array(fig)[375:1050, 475:1000]

plt.close()

Colorized gradients

In [ ]:
for i in range(7):
    fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, figsize=(5, 5), dpi=300)
    ax.scatter(coords[:, 0], coords[:, 1], coords[:, 2], c=np.sign(corrs1[i, i]) * modes1[mapping1[i]], alpha=0.5, cmap='coolwarm', edgecolor='None')
    ax.set_xlim([-0.25, 0.75])
    ax.set_ylim([-0.25, 0.75])
    ax.set_zlim([0, 1])
    ax.set_axis_off()
    plt.tight_layout(pad=0)
    figs_noise.append(figure_to_array(fig)[375:1050, 475:1000])
    plt.close()

for i in range(7):
    fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, figsize=(5, 5), dpi=300)
    ax.scatter(coords[:, 0], coords[:, 1], coords[:, 2], c=np.sign(corrs2[i, i]) * modes2[mapping2[i]], alpha=0.5, cmap='coolwarm', edgecolor='None')
    ax.set_xlim([-0.25, 0.75])
    ax.set_ylim([-0.25, 0.75])
    ax.set_zlim([0, 1])
    ax.set_axis_off()
    plt.tight_layout(pad=0)
    figs_smooth.append(figure_to_array(fig)[375:1050, 475:1000])
    plt.close()

In [ ]:
np.save('../Results/figure4_ellipse_image.npy', fig_array)
np.save('../Results/figure4_gradients_images_noise.npy',figs_noise)
np.save('../Results/figure4_gradients_images_smoothed.npy', figs_smooth)

# Part 2: Varying filtering kernel

In [ ]:
N = 2500 # Number of nodes
sigma = 0.05 # Filtering kernel width

random_ids = [1] * N + [0] * (vertices.shape[0] - N)
np.random.shuffle(random_ids)
coords = vertices[np.array(random_ids) == 1] # Node coordinates are taken randomly from 3D mesh vertices
modes_geometric = eigenmodes[np.array(random_ids) == 1, :].T # Subsampling vertices, where network nodes are located
modes_geometric = modes_geometric[1:, :] # Excluding first mode (artifact)

timeseries = np.random.normal(0, 1, (N, 10000))
for i in range(timeseries.shape[0]):
    timeseries[i] = timeseries[i]
timeseries = zscore(timeseries, axis=1)
timeseries_smoothed = zscore(spatial_smoothing(timeseries, coords, sigma=sigma), axis=1)

C = torch.corrcoef(torch.tensor(timeseries)).numpy()
C_smoothed = torch.corrcoef(torch.tensor(timeseries_smoothed)).numpy()

N_modes = 50
embedding = DiffusionMaps(n_components=N_modes)
modes1 = embedding.fit_transform(np.abs(C)).T
embedding = DiffusionMaps(n_components=N_modes, alpha=0)
modes2 = embedding.fit_transform(np.abs(C_smoothed)).T

corrs1, mapping1 = compute_mode_similarity_matrix(modes1, modes_geometric, return_mapping=True)
corrs2, mapping2 = compute_mode_similarity_matrix(modes2, modes_geometric, return_mapping=True)

In [ ]:
d = compute_distances(coords, coords)
C_theoretical = np.exp(-(d ** 2) / (4 * sigma ** 2))

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].imshow(np.abs(C_smoothed) + 1, cmap='magma', norm=LogNorm())
ax[1].imshow(np.abs(C_theoretical) + 1, cmap='magma', norm=LogNorm())

In [ ]:
triangle = np.triu_indices(C_smoothed.shape[0], 1)

In [ ]:
pearsonr(C_smoothed[triangle], C_theoretical[triangle])

In [ ]:
d_bins = np.linspace(0, 0.4, 61)

In [ ]:
corrs_real, corrs_theoretical = [], []
for i in range(len(d_bins) - 1):
    condition1 = d >= d_bins[i]
    condition2 = d < d_bins[i + 1]
    corrs_real.append(np.mean(C_smoothed[condition1 & condition2]))
    corrs_theoretical.append(np.mean(C_theoretical[condition1 & condition2]))

In [ ]:
plt.plot(d_bins[:-1], corrs_real)
plt.plot(d_bins[:-1], corrs_theoretical)

In [ ]:
np.save('../Results/figure4_corrmatrix_smoothed.npy', C_smoothed)
np.save('../Results/figure4_corrmatrix_theoretical.npy', C_theoretical)
np.save('../Results/figure4_corr_decay_real.npy', corrs_real)
np.save('../Results/figure4_corr_decay_theoretical.npy', corrs_theoretical)

# Plotting and saving 3D eigenmodes at various filter widths

In [ ]:
from visualization import *

In [ ]:
def run_simulation(N=2500, sigma=0.05, N_modes=50, T=10000):
    
    random_ids = [1] * N + [0] * (vertices.shape[0] - N)
    np.random.shuffle(random_ids)
    coords = vertices[np.array(random_ids) == 1] # Node coordinates are taken randomly from 3D mesh vertices
    modes_geometric = eigenmodes[np.array(random_ids) == 1, :].T # Subsampling vertices, where network nodes are located
    modes_geometric = modes_geometric[1:, :] # Excluding first mode (artifact)
    
    timeseries = np.random.normal(0, 1, (N, T))
    timeseries = zscore(timeseries, axis=1)
    timeseries_smoothed = zscore(spatial_smoothing(timeseries, coords, sigma=sigma), axis=1)
    
    C_smoothed = torch.corrcoef(torch.tensor(timeseries_smoothed)).numpy()
    
    embedding = DiffusionMaps(n_components=N_modes, alpha=0)
    gradients = embedding.fit_transform(np.abs(C_smoothed)).T
    
    mode_similarity, mapping = compute_mode_similarity_matrix(gradients, modes_geometric, return_mapping=True)

    return coords, mode_similarity, gradients, mapping

In [ ]:
sigma_values = [1e-10, 0.01, 0.02, 0.03, 0.04, 0.05, 0.25]

figures, corrs = [], []

for sigma in sigma_values:
    
    coords, corrs_, modes, mapping = run_simulation(sigma=sigma)

    i = 0
    v = np.percentile(np.abs(modes[mapping[i]]), 95)
    fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, figsize=(5, 5), dpi=300)
    ax.scatter(coords[:, 0], coords[:, 1], coords[:, 2], c=np.sign(corrs_[i, i]) * modes[mapping[i]], alpha=0.5, cmap='coolwarm', edgecolor='None', vmin=-v, vmax=v)
    ax.set_xlim([-0.25, 0.75])
    ax.set_ylim([-0.25, 0.75])
    ax.set_zlim([0, 1])
    ax.set_axis_off()
    plt.tight_layout(pad=0)
    fig_array = figure_to_array(fig)[375:1050, 475:1000]
    
    figures.append(fig_array)
    corrs.append(corrs_)

In [ ]:
np.save('../Results/figure4_mode_figures.npy', np.stack(figures))
np.save('../Results/figure4_corrs_lowsigma.npy', np.stack(corrs))

# Computing eigenmode-gradient correlations at varying filtering kernel widths

In [ ]:
sigma_values = np.linspace(0.01, 1.0, 100) # Linear sweep

In [ ]:
vertices = np.load('../Files/vertices_ellipse2.npy').astype('float')

order = np.argsort(vertices[:, 2]) # Ordering vertices along z-axis for more structured matrices
vertices = vertices[order]
eigenmodes = np.load('../Files/eigenmodes_ellipse2.npy')[order]

random_ids = [1] * N + [0] * (vertices.shape[0] - N)
np.random.shuffle(random_ids)
coords = vertices[np.array(random_ids) == 1] # Node coordinates are taken randomly from 3D mesh vertices

In [ ]:
N = 3000
N_modes = 50
N_runs = 10

corrs_per_sigma = []

for sigma in tqdm(sigma_values):
    
    corrs_per_run = []
    
    for _ in range(N_runs):
        
        random_ids = [1] * N + [0] * (vertices.shape[0] - N)
        np.random.shuffle(random_ids)
        coords = vertices[np.array(random_ids) == 1] # Node coordinates are taken randomly from 3D mesh vertices
        
        timeseries = np.random.normal(0, 1, (N, 2500))
        timeseries_smoothed = zscore(spatial_smoothing(timeseries, coords, sigma=sigma), axis=1)
        
        C_smoothed = torch.corrcoef(torch.tensor(timeseries_smoothed)).numpy()
                
        embedding = DiffusionMaps(n_components=N_modes)
        gradients = embedding.fit_transform(np.abs(C_smoothed)).T
        
        modes_geometric = eigenmodes[np.array(random_ids) == 1, :].T # Subsampling vertices, where network nodes are located
        modes_geometric = modes_geometric[1:, :] # Excluding first mode (artifact)
        
        corrs, mapping = compute_mode_similarity_matrix(gradients, modes_geometric, return_mapping=True)

        corrs_per_run.append(corrs)

    corrs_per_sigma.append(np.stack(corrs_per_run, axis=0))

corrs_per_sigma = np.stack(corrs_per_sigma, axis=0)

In [ ]:
avg_corrs_per_sigma = np.mean(np.abs(corrs_per_sigma), axis=1)

avg_corrs = []
for c in avg_corrs_per_sigma:
    avg_corrs.append(np.mean(np.abs(np.diag(c))))

In [ ]:
plt.plot(sigma_values, avg_corrs)

In [ ]:
np.save('../Results/figure4_smoothing_corrs_per_sigma_zoom_2.npy', corrs_per_sigma)

# Figure layout

In [ ]:
from visualization import *

In [ ]:
fig = PaperFigure(figsize=(7, 7))

fig.set_tick_length(2)
fig.set_font_size(6)
fig.add_background()

ratio = fig_array.shape[0] / fig_array.shape[1]
h = 2 
w = h / ratio
pad = 0.35
fig.add_axes('ellipse', (0, 0), w, h)
fig.add_axes('noise1', (0.4, 0.4), 0.7, 0.25)
fig.add_axes('noise2', (0.4, 1.3), 0.7, 0.25)

fig.add_axes('timeseries_noise', (w + pad , 0), 3.5, h / 2.5)
fig.add_axes('timeseries_smooth', (w + pad, h - (h / 2.5)), 3.5, h / 2.5)
fig.add_axes('corr_noise', (7 - (h / 2.5) , 0), h / 2.5, h / 2.5)
fig.add_axes('corr_smooth', (7 - (h / 2.5) , h - (h / 2.5)), h / 2.5, h / 2.5)

y = 2.5
w = 0.75
pad = (5 - 6 * w) / 5
for i in range(6):
    fig.add_axes('modes_noise{}'.format(i), (i * (w + pad), y), w, w * ratio)

for i in range(6):
    fig.add_axes('modes_smooth{}'.format(i), (i * (w + pad), y + w * ratio + 0.1), w, w * ratio)

w = h / 2.5
fig.add_axes('modecorr_noise', (7 - (h / 2.5), y), w, w)
fig.add_axes('modecorr_smooth', (7 - (h / 2.5), y + h - w), w, w)

fig.set_line_thickness(0.6)

# --------------------------------------------------------------------------------------------------

ax = fig.axes['ellipse']
ax.imshow(fig_array)
ax.axis('off')

ax = fig.axes['noise1']
ax.plot(np.random.normal(0, 1, 50), color='black', linewidth=0.6)
ax.axis('off')

ax = fig.axes['noise2']
ax.plot(np.random.normal(0, 1, 50), color='black', linewidth=0.6)
ax.axis('off')

ax = fig.axes['timeseries_noise']
ax.imshow(timeseries[::3, :1000], aspect='auto', interpolation='None', cmap='coolwarm', vmin=-2.5, vmax=2.5)
ax.set_xticks([])
ax.set_yticks([])

ax = fig.axes['timeseries_smooth']
ax.imshow(timeseries_smoothed[::3, :1000], aspect='auto', interpolation='None', cmap='coolwarm', vmin=-2.5, vmax=2.5)
ax.set_xticks([])
ax.set_yticks([])

ax = fig.axes['corr_noise']
ax.imshow(np.abs(C)[::3, ::3] + 1, cmap='Reds', norm=LogNorm(), interpolation='None')
ax.set_xticks([])
ax.set_yticks([])

ax = fig.axes['corr_smooth']
ax.imshow(np.abs(C_smoothed)[::3, ::3] + 1, cmap='Reds', norm=LogNorm(), interpolation='None')
ax.set_xticks([])
ax.set_yticks([])

for i in range(6):
    ax = fig.axes['modes_noise{}'.format(i)]
    ax.imshow(figs_noise[i])
    ax.axis('off')

for i in range(6):
    ax = fig.axes['modes_smooth{}'.format(i)]
    ax.imshow(figs_smooth[i])
    ax.axis('off')

ax = fig.axes['modecorr_noise']
ax.imshow(np.abs(corrs1), cmap='Reds', vmin=0, vmax=1)
ax.set_xticks([])
ax.set_yticks([])

ax = fig.axes['modecorr_smooth']
ax.imshow(np.abs(corrs2), cmap='Reds', vmin=0, vmax=1)
ax.set_xticks([])
ax.set_yticks([])

fig.show()

In [ ]:
fig.save('../Figures/figure4_incomplete.svg')